In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect('data/week10.db')

OperationalError: unable to open database file

In [ ]:
# 1. 고객 정보 (고객번호, 연령대, 성별)
pd.read_sql_query("""
    select *
    from CS_INFO
    LIMIT 5
""", con)

In [3]:
# 해외주식 거래내역 : 지난 1년간 고객들의 해외 주식 거래 내역
pd.read_sql_query("""
    select *
    from TRADE
    limit 5
""", con)

TypeError: read_sql_query() missing 1 required positional argument: 'con'

In [ ]:
# s&p500 지수 추종 etf(spy)의 가격변동 (벤치마크로 사용할 지수)
pd.read_sql_query("""
    select *
    from SP_PRICE
    limit 5
""", con)

In [ ]:
# 해외 주식 투자 고객의 투자 수익률 (해외 주식 투자 고객의 1년 투자 수익률입니다. )
pd.read_sql_query("""
    select *
    from YEAR_RU
    limit 5
""", con)

In [4]:
# 종목별 데이터 분석
# 월별 거래량 top10, 순매수 top10, 순매도 top 10
# 거래내역을 바탕으로 12개월 간 월별 거래량 상위 10개 종목을 알아보자

# 어떤 달에 해외주식 거래가 가장 활발했을까?
pd.read_sql_query("""
    SELECT SUBSTR(TR_DT, 1, 6) AS 월
        , SUM(매수입고금액) AS 총매수입고금액
        , SUM(매도출고금액) AS 총매도출고금액
        , SUM(매수입고금액)+SUM(매도출고금액) AS 총거래금액
    FROM TRADE
    GROUP BY 월
    ORDER BY  총거래금액 DESC -- 내림차순 정렬
    LIMIT 10 -- 상위 10개 종목
""", con)

TypeError: read_sql_query() missing 1 required positional argument: 'con'

In [ ]:
# 1년중 거래가 활발했던 2021년 1월에는 어떤 종목을 많이 거래했을까요?
pd.read_sql_query("""
    SELECT ITM_NO AS 종목명
        , SUM(매수입고금액) AS 총매수입고금액
        , SUM(매도입고금액) AS 총매도입고금액
        , SUM(매수입고금액)+SUM(매도입고금액) AS 총거래금액
    FROM TRADE 
    WHERE SUBSTR(TR_DT, 1, 6) = '202101'
    GROUP BY 종목명
    ORDER BY 총거래금액 DESC -- 내림차순 정렬
    LIMIT 10 -- 상위 10개 종목
""", con)

In [5]:
# for 문을 응용하면 월별 거래종목 상위 Top10을 한 표에서 볼 수 있다. 
months = ['202008','202009','202010','202011','202012','202101','202102','202103','202104','202105','202106','202107']
year_top10 = pd.DataFrame([]) # 빈 데이터 프레임을 하나 만들고
for m in months:
    df = pd.read_sql_query("""
        SELECT ITM_NO AS 종목명
        FROM TRADE
        WHERE SUBSTR(TR_DT, 1, 6) = '{}'
        GROUP BY 종목명
        ORDERY BY SUM(매수입고금액)+SUM(매도출고금액) DESC -- 내림차순정렬
        LIMIT 10 -- 상위 10개 종목
    """.format(m), con)
    df.rename(columns = {'종목명': m}, inplace = True) 
    year_top10 = pd.concat([year_top10, df], axis =1)

TypeError: read_sql_query() missing 1 required positional argument: 'con'

In [ ]:
year_top10

In [ ]:
# 순매수 top10 
net_pur = pd.DataFrame([])
for m in months:
    df = pd.read_sql_query("""
        SELECT ITM_NO AS 종목명
        FROM TRADE
        WHERE SUBSTR(TR_DT, 1, 6) = '{}'
        GROUP BY 종목명
        ORDER BY SUM(매수출고금액) - SUM(매도출고금액) DESC -- 순매도
        LIMIT 10 -- 상위10개 종목
    """.format(m), con)
    df.rename(columns = {'종목명': m}, inplace = True)
    net_pur = pd.concat([net_pur, df], axis=1)

In [ ]:
net_pur

In [ ]:
net_sel = pd.DataFrame([])
for m in months:
    df = pd.read_sql_query("""
    SELECT ITM_NO AS
    FROM TRADE
    WHERE SUBSTR(TR_DT, 1, 6) = '{}'
    GROUP BY 종목명
    ORDER BY SUM(매도출고금액) - SUM(매수입고금액) DESC
    LIMIT 10 -- 상위 10개종목
    """.format(m), con)
    df.rename(columns = {'종목명': m}, inplace = True)
    net_sel = pd.concat([net_sel, df], axis = 1)

In [ ]:
net_sel

In [ ]:
# 두 테이블 결합하여 분석하기 
# 고객정보로 거래량 분석하기 
# 성별에 관계 없이 테슬라가 1위
pd.read_sql_query("""
    SELECT 성별, ITM_NO AS 종목명, SUM(매수입고금액)+SUM(매도출고금액) AS 총거래금액
    FROM TRADE T
    LEFT JOIN CS_INFO C
    ON T.CS_NO = C.CS_NO
    WHERE 성별 IS NOT NULL
    GROUP BY 1,2
    ORDER BY 3 DESC
    LIMIT 10
""", con)

In [ ]:
# for 문 활용
gender = ['M','F']
gender_top10 = pd.DataFrame([])
for g in genders:
    df = pd.read_sql_query("""
        SELECT ITM_NO AS 종목명, SUM(매수입고금액)+SUM(매도출고금액) AS 총거래금액
        FROM TRADE T
        LEFT JOIN CS_INFO C
        ON T.CS_NO = C.CS_NO
        WHERE 성별 = '{}'
        GROUP BY 1
        ORDER BY 2 DESC
        LIMIT 10 -- 상위 10개 종목
    """.format(g), con)
    df.rename(columns = {'종목명':'{}_종목명'.format(g), '총거래금액': '{}_총거래금액'.format(g)}, inplace = True)
    gender_top10 = pd.concat([gender_top10, df], axis = 1)
    
    

In [ ]:
gender_top10 # 여성투자자들이 중국주식에 관심이 많았다

In [ ]:
# 연령대별로 거래량 분석해보기 
age = ['20대','30대','40대','50대','60대']
age_top10 = pd.DataFrame([])

for a in age:
    df = pd.read_sql_query("""
        SELECT ITM_NO AS 종목명
            , SUM(매도출고금액)+SUM(매수출고금액) AS 총거래금액
        FROM TRADE T
        LEFT JOIN CS_NO C
        ON T.CS_NO = C.CS_NO
        WHERE 연령 = '{}'
        GROUP BY 1
        ORDERY BY 2 DESC
        LIMIT 10
    """.format(a), con)
    df.rename(colums = {'종목명':'{}_종목명'.format(a),'총거래금액': '{}_총거래금액'.format(a)}, inplace = True)
    age_top10 = pd.concat([age_top10, df], axis =1)

In [ ]:
age_top10

In [ ]:
# 투자수익률과 결합하여 분석하기
# 고객의 1년 투자 수익률을 기준으로 거래량을 파악해보자
# 수익률이 높은 투자 고수들은 다른 고객과 차별화된 종목을 골랐을까

# 가장 먼저 수익률 구간을 나누어 주는 작업이 필요
# 우선 두 데이블을 조인해야함
pd.read_sql_query("""
    SELECT T.*, RT AS 수익률
    FROM TRADE T
    LEFT JOIN YEAR_RT R
    ON T.CS_NO = R.CS_NO
    WHERE 수익률 IS NOT NULL
    LIMIT 10
""", con)

In [ ]:
# sp500 etf 보다 투자 수익률이 좋으면 고수
# spy 일별 종가 데이터
pd.read_sql_query("""
    SELECT *
    FROM SP_PRICE
    WHERE BAS_DT IN ('20200803','20210730')
""", con)

# 1년동안의 수익률은 (438.51 - 328.79)/328.79 = 33.37%

In [ ]:
# 투자자들의 수익률을 구간으로 나눔
pd.read_sql_query("""
    SELECT T.*
        , RT AS 
        , CASE WHEN RT > 100 THEN '초고수'
               WHEN RT > 33.37 THEN '고수'
               WHEN RT < 0 THEN '하수'
        ELSE '중수' END AS 수익률_구간
        
    FROM TRADE T
    LEFT JOIN YEAR_RT R
    ON T.CS_NO = R.CS_NO
    WHERE 수익률 IN NOT NULL
    LIMIT 10
""", con)

In [ ]:
# for 문, with 문 활용
ranks = ['초고수','고수','중수','하수']
rank_top10 = pd.DataFrame([])
for r in ranks:
    df = pd.read_sql_query("""
        WITH MASTER AS (
            SELECT T.*
                , RT AS 수익률
                , CASE WHEN RT>100 THEN '초고수'
                       WHEN RT >33.37 THEN '고수'
                       WHEN RT < 0 THEN '하수'
                ELSE '중수' END AS 수익률_구간
            FROM TRADE T
            LEFT JOIN YEAR_RT R
            ON T.CS_NO = R.CS_NO
            WHERE 수익률 IS NOT NULL
        )  -- MASTER 라는 테이블 정의 
        
        SELECT ITM_NO AS 종목명 , SUM(매수입고금액)+SUM(매도입고금액) AS 총거래금액
        FROM MASTER M
        WHERE 수익률_구간 = '{}'
        GROUP BY 1
        ORDER BY 2 DESC
        LIMIT 10
    """.format(r), con)
    
    df.rename(columns = {'종목명':'{}_종목명'.format(r),'총거래금액':'{}_총거래금액'}, inplace = True)
    rank_top10 = pd.concat([rank_top10, df], axis = 1)

In [ ]:
rank_top10